In [1]:
import pandas as pd
import numpy as np
import scipy

In [2]:
dataset = pd.read_csv('data/LUMEN_DS.csv', sep='|', quotechar='"', encoding='UTF-16LE')
#negative_gm = dataset[dataset['GM%'] < 0]
dataset.shape
test_data = dataset.iloc[0].to_frame().copy(deep=True)

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (16,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
from datetime import datetime
import calendar

def transform_order_date(order_date):
    #order_date = order_date_column.values
    order_date = order_date.apply(datetime.fromisoformat)
    day_of_week = []
    day_of_month = []
    day_of_year = []
    for date in order_date:
        dow = date.weekday()
        date_day = date.day-1
        doy = date.timetuple().tm_yday - 1 # starts from 1
        month_length = calendar.monthrange(date.year, date.month)[1]
        year_length = 365 if not calendar.isleap(date.year) else 366
        day_of_week.append(2*np.pi*dow / 7)
        day_of_month.append(2*np.pi*date_day / month_length)
        day_of_year.append(2*np.pi*doy / year_length)
    dow = np.array(day_of_week)
    dom = np.array(day_of_month)
    doy = np.array(day_of_year)
    return np.cos(dow), np.sin(dow), np.cos(dom), np.sin(dom), np.cos(doy), np.sin(doy)

In [4]:
from sklearn_pandas import DataFrameMapper

uninformative_columns = [
    'CustomerID',
     'Item Code',
     'Invoice #',
     'Order #',
     'Invoiced price (TX)'
]

NA_THRESHOLD = 0.1
na_share = dataset.isna().mean()
na_columns = na_share[na_share > NA_THRESHOLD].index.tolist()

#all_columns = make_column_selector('.*')


mapper = DataFrameMapper([], default=None, drop_cols=uninformative_columns+na_columns, df_out=True)

In [5]:
from sklearn.preprocessing import FunctionTransformer

def time_features(date):
#     print(type(date))
    dow_x, dow_y, dom_x, dom_y, doy_x, doy_y = transform_order_date(date['Order Date'])
    date['dow_x'] = dow_x
    date['dow_y'] = dow_y
    date['dom_x'] = dom_x
    date['dom_y'] = dom_y
    date['doy_x'] = doy_x
    date['doy_y'] = doy_y
    return date #date.drop(['Order Date'], axis=1)


#xxx = FunctionTransformer(transform_order_date)
date_mapper = DataFrameMapper([(['Order Date'], FunctionTransformer(time_features))], default=None, input_df=True, df_out=True)
#dataset = date_mapper.fit_transform(dataset)

In [6]:
from sklearn.pipeline import Pipeline

#dataset = pd.read_csv('data/LUMEN_DS.csv', sep='|', quotechar='"', encoding='UTF-16LE')
#negative_gm = dataset[dataset['GM%'] < 0]
#dataset.shape

class Dropper():
    def fit(self, *args):
        return self

    def transform(self, x):
        return x.dropna()

lumen_pipeline = Pipeline([
    ('mapper', mapper),
    ('dropper', Dropper()),
    ('date_mapper', date_mapper),
    ('date_dropper', DataFrameMapper([], default=None, drop_cols=['Order Date'], input_df=True, df_out=True)),
    ('last', 'passthrough'),
])
lumen_pipeline.fit_transform(dataset)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

,Order Date_0,Order Date_1,Order Date_2,Order Date_3,Order Date_4,Order Date_5,Order Date_6,Manufacturing Region,Manufacturing Location Code,Intercompany,...,Make vs Buy,Invoice Date,Invoice Line #,Order Line #,Invoiced qty (shipped),Ordered qty,Invoiced price,Cost of part,GM%,# of unique products on a quote
0,2019-08-05,1.000000,0.000000,0.688967,0.724793,-0.838280,-0.545240,North America,N7,YES,...,BUY,2019-08-06,18645771,15791584.0,499.0,499.0,0.39,0.4173,-0.07,1.0
1,2019-11-24,0.623490,-0.781831,0.104528,-0.994522,0.793572,-0.608477,North America,N7,NO,...,BUY,2019-11-26,18670413,15806968.0,1000.0,1000.0,0.14,0.0257,0.816429,3.0
3,2020-03-10,0.623490,0.781831,-0.250653,0.968077,0.376728,0.926324,Asia,N13,NO,...,BUY,2020-03-20,18688647,15818226.0,2000.0,5000.0,0.2,0.0,1.0,1.0
4,2020-03-10,0.623490,0.781831,-0.250653,0.968077,0.376728,0.926324,Asia,N13,NO,...,BUY,2020-03-20,18688639,15818226.0,1230.0,5000.0,0.2,0.0,1.0,1.0
5,2020-03-10,0.623490,0.781831,-0.250653,0.968077,0.376728,0.926324,Asia,N13,NO,...,BUY,2020-03-17,18687466,15818226.0,1770.0,5000.0,0.2,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294956,2020-05-08,-0.900969,-0.433884,0.151428,0.988468,-0.586396,0.810025,North America,N7,NO,...,BUY,2020-05-11,14731882,18486911.0,32.0,32.0,36.32,22.5,0.380507,5.0
1294958,2019-03-15,-0.900969,-0.433884,-0.954139,0.299363,0.309017,0.951057,North America,N15,NO,...,MANUFACTURED,2019-03-19,9880134,14796498.0,15.0,15.0,7.0,0.75,0.892857,1.0
1294959,2019-06-04,0.623490,0.781831,0.809017,0.587785,-0.882048,0.471160,North America,N7,NO,...,BUY,2019-08-12,18647035,15783555.0,10000.0,10000.0,0.0702,0.045,0.358974,1.0
1294960,2019-06-05,-0.222521,0.974928,0.669131,0.743145,-0.890028,0.455907,North America,N7,NO,...,BUY,2019-08-13,18646308,15783745.0,3000.0,3000.0,1.977,1.1336,0.426606,1.0


In [7]:
lumen_pipeline

Pipeline(steps=[('mapper',
                 DataFrameMapper(default=None, df_out=True,
                                 drop_cols=['CustomerID', 'Item Code',
                                            'Invoice #', 'Order #',
                                            'Invoiced price (TX)',
                                            'Product group',
                                            'Price last modified date in the '
                                            'ERP',
                                            'Sales Channel - Internal',
                                            'Sales Channel - External',
                                            'Sales Channel - Grouping',
                                            'Material cost of part',
                                            'Labor cost of part',
                                            'Overhead cost of part'],
                                 features=[])),
                ('dropper', <__main__.Dropper o

In [8]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

cluster_drop = DataFrameMapper([], default=None, drop_cols=[
    'Manufacturing Location Code',
    'Born on date',
    'Make vs Buy',
    'Customer First Invoice Date',
    'Born on date',
    'Make vs Buy',
    'Invoice Date',
    'Invoice Line #',
    'Price last modified date in the ERP',
    'Order Line #',
    'Invoiced qty (shipped)'], input_df=True, df_out=True)

onehot_cols = [
 "Manufacturing Region",
 "Intercompany",
 "Customer industry",
 "Customer Region",
 "Top Customer Group",
 "Product family"
]



dummies = DataFrameMapper([
    (onehot_cols, OneHotEncoder(sparse=False))
], default=None)

In [9]:
from sklearn.cluster import KMeans

KNEE = 88

cluster_pipeline = Pipeline([
    ('features', lumen_pipeline),
    ('cluster_drop', cluster_drop),
    ('dummies', dummies),
    ('last', 'passthrough'),
    #('dim_red', PCA(n_components=0.85)),
    #('model', KMeans(n_clusters=KNEE))
    
])
cluster_pipeline.fit(dataset)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

Pipeline(steps=[('features',
                 Pipeline(steps=[('mapper',
                                  DataFrameMapper(default=None, df_out=True,
                                                  drop_cols=['CustomerID',
                                                             'Item Code',
                                                             'Invoice #',
                                                             'Order #',
                                                             'Invoiced price '
                                                             '(TX)',
                                                             'Product group',
                                                             'Price last '
                                                             'modified date in '
                                                             'the ERP',
                                                             'Sales Channel - '
                     

In [18]:
#cluster_pipeline.transform(test_data)

KeyError: 'Order Date'